In [0]:
pip install kafka-python

Python interpreter will be restarted.
Collecting kafka-python
 Using cached kafka_python-2.0.2-py2.py3-none-any.whl (246 kB)
Installing collected packages: kafka-python
Successfully installed kafka-python-2.0.2
Python interpreter will be restarted.

In [0]:
pip install avro

Python interpreter will be restarted.
Collecting avro
 Downloading avro-1.10.1.tar.gz (68 kB)
Building wheels for collected packages: avro
 Building wheel for avro (setup.py): started
 Building wheel for avro (setup.py): finished with status 'done'
 Created wheel for avro: filename=avro-1.10.1-py3-none-any.whl size=96804 sha256=a56f6ec6e10a6fec19e4c8fe4b48cc15ae232404ebe515868a27e44e5fa8ea73
 Stored in directory: /root/.cache/pip/wheels/97/a2/d2/fc16baff96b000ffffd8d359730fa92239279ab34b0dd66104
Successfully built avro
Installing collected packages: avro
Successfully installed avro-1.10.1
Python interpreter will be restarted.

In [0]:
%scala
System.setProperty("ssl.ca.location","dbfs:/FileStore/tables/cloudkarafka.ca");

res0: String = dbfs:/FileStore/tables/cloudkarafka.ca

In [0]:
%scala
System.clearProperty("java.security.auth.login.config");
System.clearProperty("sasl.jaas.config");
System.setProperty("java.security.auth.login.config", "dbfs:/FileStore/tables/cloudkarafka.jaas");
System.setProperty("sasl.jaas.config", "dbfs:/FileStore/tables/cloudkarafka.jaas");
println("auth.lang:" + System.getProperty("java.security.auth.login.config"));
println("sasl.jaas:" + System.getProperty("sasl.jaas.config"));

spark.sparkContext.addFile("dbfs:/FileStore/tables/cloudkarafka.jaas")


auth.lang:dbfs:/FileStore/tables/cloudkarafka.jaas
sasl.jaas:dbfs:/FileStore/tables/cloudkarafka.jaas

In [0]:
from pyspark.sql import Row
from pyspark.sql.avro.functions import from_avro, to_avro

topic = "yrmfqh3q-avro-input"

print("====================== Read CSV ======================")
filePath = "/FileStore/tables/customer_applications.csv"
read_csv_df = spark.read.format('csv').options(header='true').options(inferSchema='true').load(filePath)
print("read_csv_df:", read_csv_df)
read_csv_df.show()

def convertToAVRO(in_df):
  print("convertToAVRO::in_df:", in_df)
  in_df.show()
  return in_df

#easy way
avro_write_loc = "/tmp/csv2avro"
read_csv_df.write.format("avro").mode("overwrite").save(avro_write_loc)
print("STEP-1:COMPLETED CSV TO AVRO USING DF. NO-TRANSFORMATION")

#Ideal way:
#Convert to AVRO
#avroDf = read_csv_df.transform(convertToAVRO)
#print("avroDf:", avroDf)
#avroDf.show()
#avroDf.count()

print("STEP-2:START READ AVRO AND PUSH TO KAFKA")
read_avro_df = spark.read.format('avro').load(avro_write_loc + "/part*.avro")
print("read_avro_df:", read_avro_df)
read_avro_df.show()

from kafka import KafkaProducer

def publishToKafka2(rows):
  print("publishToKafka2:", type(rows), rows)
  producer = KafkaProducer(
    bootstrap_servers=['omnibus-01.srvs.cloudkafka.com:9094','omnibus-03.srvs.cloudkafka.com:9094','omnibus-02.srvs.cloudkafka.com:9094'],
    client_id= "Kafka-Producer",
    security_protocol="SASL_SSL",
    sasl_mechanism="SCRAM-SHA-256",
    sasl_plain_username="yrmfqh3q",
    sasl_plain_password="WfKJEYL_hwNUHVU8laSbj6gXkPIw_xuc")
  for row in rows:
    print("publishToKafka2::row:", type(row), row)
    #producer.send(topic, {"message", str(row.asDict())} )
    print("row:", type(row), row)
    producer.send(topic, key=b'message', value=bytes(str(row.asDict()), "utf-8"))
  producer.flush() #Flush after queuing up all the rows.

read_avro_df.foreachPartition(publishToKafka2)


====================== Read CSV ======================
read_csv_df: DataFrame[id: int, cust_id: int, app_id: int, active: int]
+---+-------+------+------+
 id|cust_id|app_id|active|
+---+-------+------+------+
 1| 1| 1| 1|
 2| 1| 2| 1|
 2| 2| 1| 1|
 2| 2| 2| 1|
 3| 3| 1| 1|
 4| 3| 2| 0|
+---+-------+------+------+

STEP-1:COMPLETED CSV TO AVRO USING DF. NO-TRANSFORMATION
STEP-2:START READ AVRO AND PUSH TO KAFKA
read_avro_df: DataFrame[id: int, cust_id: int, app_id: int, active: int]
+---+-------+------+------+
 id|cust_id|app_id|active|
+---+-------+------+------+
 1| 1| 1| 1|
 2| 1| 2| 1|
 2| 2| 1| 1|
 2| 2| 2| 1|
 3| 3| 1| 1|
 4| 3| 2| 0|
+---+-------+------+------+